# Assessment template for data p01

will be using the Constraint satisfaction problem algo (CSP)

In [6]:
# Install contraint lybrary
%pip install python-constraint

from constraint import *

Note: you may need to restart the kernel to use updated packages.


In [7]:
# If using Colab to upload files, if locally wont work

# from google.colab import files
# # Expected name for this example `p01_dataset_8.txt`

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))



Define the file to use as dataSet

In [8]:
# Expected name for this example `p01_dataset_8`
file_path = input("Enter the path path") or "p01_dataset_8.txt";

### Data structures definition

In [9]:
class GeneralInformation:
	def __init__(self, projects, jobs, horizon, 
							renewable_resources, nonrenewable_resources, doubly_constrained_resources):
		self.projects = projects
		self.jobs = jobs
		self.horizon = horizon
		self.resources = {
			"renewable": renewable_resources,
			'nonrenewable' : nonrenewable_resources,
			'doubly_constrained': doubly_constrained_resources
		}
	
	def __str__(self):
		return "\n".join([
			f"Projects: {self.projects}",
			f"Jobs: {self.jobs}",
			f"Horizon: {self.horizon}",
			f"Resources: {self.resources}",
		]);


class ProjectSummary:
	def __init__(self, project_number, jobs, release_date, due_date, tardiness_cost, mpm_time):
		self.project_number = project_number
		self.jobs = jobs
		self.release_date = release_date
		self.due_date = due_date
		self.tardiness_cost = tardiness_cost
		self.mpm_time = mpm_time

	def __str__(self):
		return "\n".join([
			f"  - Project Number: {self.project_number}",
			f"\tJobs: {self.jobs}",
			f"\tRelease Date: {self.release_date}",
			f"\tDue Date: {self.due_date}",
			f"\tTardiness Cost: {self.tardiness_cost}",
			f"\tMPM Time: {self.mpm_time}",
		]);


class Job:
	def __init__(self, job_number, successors):
		self.job_number = job_number
		self.successors = successors

	def get_successors_count(self):
		return len(self.successors);

	def __str__(self):
		return "\n".join([
			f"  - Job Number: {self.job_number}",
			f"\tSuccessors Count: {self.get_successors_count()}",
			f"\tSuccessors: {', '.join(map(str, self.successors)) if self.successors else 'None'}",
		]);


class DurationResource:
	def __init__(self, job_number, mode, duration, resources):
		self.job_number = job_number
		self.mode = mode
		self.duration = duration
		self.resources = resources

	def __str__(self):
		return "\n".join([
			f"  - Job Number: {self.job_number}",
			f"\tMode: {self.mode}",
			f"\tDuration: {self.duration} days",
			f"\tResources: {', '.join([f'{key}: {value}' for key, value in self.resources.items()]) if self.resources else 'None'}",
		]);


class ResourceAvailability:
	def __init__(self, resource_name, quantity):
		self.resource_name = resource_name
		self.quantity = quantity


	def __str__(self):
		return f"  - Resource: {self.resource_name}\n\tQuantity: {self.quantity}";


In [10]:
# Definition for data loaded
class ProjectData:
	def __init__(self):
		self.general_info = None
		self.projects_summary = []
		self.precedence_relations = []
		self.durations_resources = []
		self.resource_availability = {} 

	def __str__(self):
		result = []

		# General Info
		result.append("General Information:")
		if self.general_info:
				result.append(str(self.general_info))
		else:
				result.append("  None")
		
		# Projects Summary
		result.append("\nProjects Summary:")
		if self.projects_summary:
				for summary in self.projects_summary:
						result.append(str(summary))
		else:
				result.append("  None")
		
		# Precedence Relations
		result.append("\nPrecedence Relations:")
		if self.precedence_relations:
				for relation in self.precedence_relations:
						result.append(str(relation))
		else:
				result.append("  None")
		
		# Durations and Resources
		result.append("\nDurations and Resources:")
		if self.durations_resources:
				for resource in self.durations_resources:
						result.append(str(resource))
		else:
				result.append("  None")
		
		# Resource Availability
		result.append("\nResource Availability:")
		if self.resource_availability:
				for name, resource in self.resource_availability.items():
						result.append(str(resource))
		else:
				result.append("  None")
		
		return "\n".join(result)


In [15]:
def parse_data()-> ProjectData:
	data = ProjectData()
	# Track the file section
	section = None

	with open(file_path, 'r') as file:
		for line in file:
			line = line.strip()

			# Skip ornament and invalid
			if(line.startswith('**') or not line):
				continue;

			if(line.startswith('#')):
				# Match the file section
				match line:
					case "#General Information": 		section = "general_info"; continue;
					case "#Projects summary": 			section = "projects_summary"; continue;
					case "#Precedence relations": 	section = "precedence_relations"; continue;
					case "#Duration and resources":	section = "durations_resources"; continue;
					case "#Resource availability":	section = "resource_availability"; continue;

			match section:
				case "general_info":
					if "projects:" in line:
						data.general_info = GeneralInformation(
							int(line.split(':')[1].strip()),
							jobs=0, horizon=0,
							renewable_resources=0, nonrenewable_resources=0, doubly_constrained_resources=0
						)
					elif "jobs" in line:
						data.general_info.jobs = int(line.split(':')[1].strip());
					elif "horizon" in line:
						data.general_info.horizon = int(line.split(':')[1].strip());
					elif line.startswith("- renewable"):
						data.general_info.resources["renewable"] = int(line.split(":")[1].split()[0].strip());
					elif line.startswith("- nonrenewable"):
						data.general_info.resources["nonrenewable"] = int(line.split(":")[1].split()[0].strip());
					elif line.startswith("- doubly constrained"):
						data.general_info.resources["doubly_constrained"] = int(line.split(":")[1].split()[0].strip());

				case "projects_summary":
					# Skip header line
					if(line.startswith("pronr.")):
						continue;
					
					splits = line.split();
					if splits:
						data.projects_summary.append(ProjectSummary(
							project_number 	= int(splits[0]),
							jobs 						= int(splits[1]),
							release_date 		= int(splits[2]),
							due_date 				= int(splits[3]),
							tardiness_cost 	= int(splits[4]),
							mpm_time 				= int(splits[5])
						));

				case "precedence_relations":
					# Skip header line
					if(line.startswith("#jobnr.")):
						continue;

					splits = line.split()
					if(splits):
						data.precedence_relations.append(Job(
							job_number = int(splits[0]),
							successors = list(map(int, splits[3:]))
						));

				case "durations_resources":
					# Skip header line
					if(line.startswith("#jobnr.")):
						continue

					splits = line.split()
					if(splits):
						resourcesData = {}
						for i in range(len(splits) - 3):
							resourcesData[f"R{i+1}"] = int(splits[i+3]);

						data.durations_resources.append(DurationResource(
							job_number = int(splits[0]),
							mode = int(splits[1]),
							duration = int(splits[2]),
							resources = resourcesData
						));

				case "resource_availability":
					# Skip header line
					if(line.startswith("#resource")):
						continue;

					splits = line.split();
					if(splits):
						name = splits[0];
						
						data.resource_availability[name] = ResourceAvailability(
							resource_name = name,
							quantity = int(splits[1])
						);

	return data;

In [16]:
pData = parse_data()
print(pData)

General Information:
Projects: 1
Jobs: 8
Horizon: 20
Resources: {'renewable': 2, 'nonrenewable': 0, 'doubly_constrained': 0}

Projects Summary:
  - Project Number: 1
	Jobs: 8
	Release Date: 0
	Due Date: 11
	Tardiness Cost: 0
	MPM Time: 11

Precedence Relations:
  - Job Number: 1
	Successors Count: 2
	Successors: 2, 3
  - Job Number: 2
	Successors Count: 1
	Successors: 4
  - Job Number: 3
	Successors Count: 1
	Successors: 4
  - Job Number: 4
	Successors Count: 0
	Successors: None
  - Job Number: 5
	Successors Count: 2
	Successors: 6, 7
  - Job Number: 6
	Successors Count: 1
	Successors: 8
  - Job Number: 7
	Successors Count: 1
	Successors: 8
  - Job Number: 8
	Successors Count: 0
	Successors: None

Durations and Resources:
  - Job Number: 1
	Mode: 1
	Duration: 2 days
	Resources: R1: 1, R2: 0
  - Job Number: 2
	Mode: 1
	Duration: 3 days
	Resources: R1: 0, R2: 1
  - Job Number: 3
	Mode: 1
	Duration: 4 days
	Resources: R1: 0, R2: 1
  - Job Number: 4
	Mode: 1
	Duration: 1 days
	Resources: R

# Defining the solver

### Problem BreakDown

- Variables: Each job can be seen as a variable. The value of this variable will be the start time of that job.
- Domains: The domain for each job variable will be the possible start times, considering the project’s time horizon.
- Constraints:
	- Precedence Constraints: A job cannot start before its predecessors are finished (if job A precedes job B, then job A’s end time must be before job B starts).
	- Resource Constraints: The number of resources (e.g., designers, programmers) is limited. So, at any given time, the total resources used by all jobs must not exceed the available resources.
	- Duration Constraints: Each job has a specific duration, so its end time is determined by its start time and duration.

In [14]:
# Precedence constrain function, a new job should only start if the precedents ended
def precendence_cstr(start, duration, successor_start)-> bool:
	return start + duration <= successor_start;

In [ ]:
def resource_cstr(job_start, job_number):
	# Find the job's resource requirements and duration in the project data
	job_resources = next((dr.resources for dr in pData.durations_resources if dr.job_number == job_number), {})
	job_duration = next((dr.duration for dr in pData.durations_resources if dr.job_number == job_number), 0)

	# Check resource availability for each required resource
	for resource, quantity_needed in job_resources.items():
			# Look up the total available quantity for this resource in the project data
			available_quantity = pData.resource_availability.get(resource, 0)
			
			# If the required quantity for this job exceeds availability, return False
			if quantity_needed > available_quantity:
					return False

	# If all required resources are available, return True
	return True

In [ ]:
def define_problem()-> Problem:
	problem = Problem();

	# Define variables for each job, and the domain are the available time on the horizon (20 days)
	for job in pData.precedence_relations:
		number = job.job_number;
		problem.addVariable(f"start_t_{number}",range(pData.general_info.horizon + 1));

	for job in pData.precedence_relations:
		for sucessor in job.successors:
			number = job.job_number;
			job_start = f"start_t_{number}";
			successor_start = f"start_t_{sucessor}";

			job_duration = next((dr.duration for dr in pData.durations_resources if dr.job_number == number), 0)

			problem.addConstraint(precendence_cstr, [job_start, job_duration, successor_start])


	for job in pData.precedence_relations:
		job_number = job.job_number
		job_start = f"start_t_{job_number}"

    # Add resource constraint to ensure resources are available when the job starts	
		problem.addConstraint(resource_constraint, [job_start, job_number])


	return problem;